<a href="https://colab.research.google.com/github/juan-holguin/Senales_Sistemas/blob/main/Copia_de_Parcial1_SyS_2024_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parcial 1: Señales y Sistemas 2024-II

 ## Profesor: Andrés Marino Álvarez Meza, Ph.D.


## Departamento de Ingeniería Eléctrica, Electrónica, y Computación
## Universidad Nacional de Colombia - sede Manizales

# Instrucciones

-- Para recibir el crédito total, sus respuestas deben estar justificadas de manera clara, detallada y concreta, mostrando los procedimientos y razonamientos paso a paso.

-- Está permitido el uso de herramientas de inteligencia artificial (IA). Si las utiliza, por favor declare explícitamente cómo fueron empleadas en la resolución de cada pregunta. Incluya los prompts (consultas) y las iteraciones realizadas con las IA durante el desarrollo del parcial.

-- La entrega del parcial debe realizarse antes de las 23:59 del 5 de diciembre de 2024 al correo electrónico amalvarezme@unal.edu.co mediante un enlace de GitHub.

-- Los códigos deben estar debidamente comentados en las celdas correspondientes y explicados en celdas de texto (markdown). Los códigos que no incluyan comentarios ni discusiones no serán considerados en la evaluación final.

# Pregunta 1 (valor 2.5 puntos)

Cuál es la señal obtenida en tiempo discreto al utilizar un conversor análogo digital de 5 bits con frecuencia de muestreo de $5kHz$, aplicado a la señal continua $x(t) = 0.3 \cos(1000\pi t-\pi/4) +
0.6 \sin(2000\pi t) + 0.1 \cos(11000\pi t-\pi)$?. Realizar la simulación del proceso de digitalización incluyendo al menos 3 ciclos de la señal $x(t)$.

En caso de que la digitalización no sea apropiada, diseñe e implemente un conversor adecuado para la señal estudiada. El convesor debe permitir configurar la cantidad de bits y la frecuencia de muestreo, indicándole al usuario si dicha frecuencia es apropiada o no, y graficar la señal continua, discreta y digital.

In [ ]:
import numpy as np # importamos la libreria numpy para los calculos y creacion de señales
import matplotlib.pyplot as plt # importamos la libreria matpltlib para obtener las graficas

A_1 = 0.3
A_2 = 0.6
A_3 = 0.1
# identificamos las amplitudes de cada señal

W_1 = 1000*np.pi
W_2 = 2000*np.pi
W_3 = 11000*np.pi

#identificamos cada uno de los omegas de la señal

Phase_1 = -np.pi/4
Phase_2 = 0
Phase_3 = -np.pi

#identificamos cada uno de los Phi (fase)

T_1 = 2*np.pi/W_1
T_2 = 2*np.pi/W_2
T_3 = 2*np.pi/W_3

#identificamos cada uno de los T (periodos)

F_1 = W_1/(2*np.pi)
F_2 = W_2/(2*np.pi)
F_3 = W_3/(2*np.pi)

#identificamos cada uno de los F (frecuencias)

T_max = 1 / min(F_1, F_2, F_3) # periodo del componente de menor frecuencia (tomada de chat gpt)

Fs = 50000 #frecuencia de muestreo
Ts = 1/Fs # periodo de muestreo
ti = 0*Ts  # tiempo inicial
tf = 3*T_max # tiempo final
tv = np.arange(ti,tf,1/Fs) # vector de datos de tiempo

x = A_1*np.cos(W_1*tv+Phase_1) + A_2*np.sin(W_2*tv+Phase_2) + A_3*np.cos(W_3*tv+Phase_3)

print(f'F_1={F_1} Hz F_2={F_2} F_3={F_3}Hz Fs={Fs} Hz')

plt.plot(tv,x, 'b', label='$X(t)$')
plt.xlabel('tiempo [s]')
plt.ylabel('$X(t)$')
plt.legend()
plt.show()


In [ ]:
def my_ceropen(x,ymin = -5, ymax = 5): # funcion de cero y pendiente (tomada de ejercicios anteriores)

  xmax = max(x) # x.max()
  xmin = min(x) # x.min()
  m = (ymax - ymin)/(xmax - xmin)
  c = ymin - m*xmin
  yv = m*x + c
  return yv
xcp = my_ceropen(x,ymin=-3.3,ymax=3.3) #el rango del conversor A/D va a ser de -3.3 a 3.3
plt.plot(xcp)
plt.xlabel('tiempo [s]')
plt.ylabel('$A\cos[\omega t_k]$')
plt.show()


In [ ]:
from scipy.spatial.distance import cdist

nbits = 5
rmin = -3.3
rmax = 3.3
ve = np.linspace(rmin,rmax,2**nbits) # (rmax-rmin)/(2^#bits - 1) -> si se quiere incluir rmin y rmax

#distancias y cero pendiente a vector cuantizacion
d = cdist(xcp.reshape(-1,1),ve.reshape(-1,1)) #se utiliza reshape(-1,1) para generar vectores columna como lo pide cdist
d.shape
ve

In [ ]:
def my_cuantizador(yn, vq) : # funcion de cuantizador (tomada de un ejercicio anterior)
  Ne = vq.shape[0] #tamaño vector de estados
  dn = cdist(yn.reshape(-1,1),vq.reshape(-1,1))#distancia yn a vector estados, reshape(-1,1) asegura vectores columna para poder utilizar cdist
  ind = np.argmin(dn) #posición distancia min
  return vq[ind]

In [ ]:
ind = np.argmin(d,axis=1) #el parámetro axis = 1 indica que busca la posición a lo largo de las columnas del elemento más pequeño en cada fila
ind

In [ ]:
for i in range(xcp.shape[0]):
   if i==0: #primera muestra
     yq = my_cuantizador(xcp[i],ve)
   else:
     yq = np.append(yq,my_cuantizador(xcp[i],ve)) #apilar datos cuantizando si i>0

In [ ]:
plt.plot(tv,xcp,c='b', label='xcp_cont')#cs continua

plt.stem(tv,xcp, label='xcp_dis')#cs discreta

#estados de cuantización

for v in ve:
  plt.plot(tv,v*np.ones(len(tv)))

# graficar señal Digital (discreta + cuantizada)

plt.plot(tv,yq,c='y',marker='o',label='xcp_cuant', linewidth=2)

plt.legend()
plt.grid()
plt.xlabel('t')
plt.ylabel('Amplitud')
plt.show()

# Pregunta 2 (valor 2.5 puntos)

Se dispone de un sistema modelado como una "caja negra" (ver celdas de código). Su tarea es analizar y comprobar mediante simulaciones si el sistema cumple con las propiedades de linealidad e invariancia en el tiempo. En caso de que el sistema sea lineal e invariante con el tiempo, determine su respuesta al impulso y utilice esta respuesta para calcular la salida del sistema ante la siguiente señal:

$x[n] = \sin[100 \pi n ] + \sin[600 \pi n]$

In [ ]:
# cargar sistema
FILEID = "1J9rhh0wWHZSBd8XmWGt1ZpCsMDuoUFmm"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O P1_model.zip && rm -rf /tmp/cookies.txt
!unzip -o P1_model.zip
!dir

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import firwin, freqz, lfilter, filtfilt

class My_System():
    def __init__(self, fs=1000, cutoff=100, numtaps = 10):
        self.fs = fs
        self.cutoff = 100
        self.numtaps = numtaps
    def create_(self):
    # Diseño del filtro FIR pasa baja
        self.fir_coeff = firwin(self.numtaps, self.cutoff, fs=self.fs, pass_zero='lowpass')

    def predict(self,signal):
       return lfilter(self.fir_coeff, 1.0, signal)

    def plot_(self):
      # Visualización de la respuesta en frecuencia
      w, h = freqz(self.fir_coeff, worN=8000, fs=self.fs)
      plt.figure()
      plt.plot(w, 20 * np.log10(abs(h)), 'b')
      plt.title('Respuesta en Frecuencia del Filtro FIR Pasa Baja')
      plt.xlabel('Frecuencia (Hz)')
      plt.ylabel('Magnitud (dB)')
      plt.grid()
      plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from P1_model import system_
from scipy.signal import firwin, freqz, lfilter, filtfilt

#sistema pregunta 2
my_system = system_.My_System()
my_system.create_()
fs = my_system.fs #frecuencia de muestreo
t = np.arange(-0.01, 0.02, 1/fs)  # Tiempo
signal_u = np.heaviside(t,1) # función heaviside
y_u = my_system.predict(signal_u)


# Visualización de las señales
fig, axs = plt.subplots(2,1)
axs[0].stem(t, signal_u, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_u, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

In [ ]:
# prueba de la propiedad de superposicion
n = np.arange(0, 100)
x1 = np.sin(10*np.pi*t)
x2 = np.sin(60*np.pi*t)
y1 = my_system.predict(x1)
y2 = my_system.predict(x2)
x_sum = x1 + x2
y_sum = my_system.predict(x_sum)
# podemos decir que cumple con la propiedad de superposicion

fig, axs = plt.subplots(2,1)
axs[0].stem(t, y_sum, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y1 + y2, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

In [ ]:
# prueba de la propiedad de homogeneidad
a = 4 # escalar
y3 = my_system.predict(a*x1)
y4 = a*y1
# podemos decir que cumple con la propiedad de homogeneidad

fig, axs = plt.subplots(2,1)
axs[0].stem(t, y3, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y4, label='Señal salida')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

In [ ]:
# prueba de que es invariante en el tiempo
s = np.cos(600*np.pi*t)
y_original = my_system.predict(s)
desp = 10
s_desp = np.roll(s, desp)
y_desp = my_system.predict(s_desp)
y_desp_original = np.roll(y_original, desp) # np.roll es el encargado de hacer el desplazamiento en un array
# podemos decir que es invariante en el tiempo

fig, axs = plt.subplots(2,1)
axs[0].stem(t, y_desp, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(t,y_desp_original, label='Señal desplazada')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()

In [ ]:
N = 50 # numero de muestras
delta = np.zeros(N)
delta[0] = 1 # impulso unitario

h = my_system.predict(delta)

# señal de entrada
s_in = np.sin(100*np.pi*n/fs) + np.sin(600*np.pi*n/fs)

s_conv = np.convolve(s_in, h, mode='same') # convolucion

fig, axs = plt.subplots(2,1)
axs[0].stem(n, s_in, label='Señal de entrada')
axs[0].set_xlabel('Tiempo (s)')
axs[0].set_ylabel('Amplitud')
axs[0].legend()
axs[0].grid()
axs[1].stem(n, s_conv, label='Convolucion')
axs[1].set_xlabel('Tiempo (s)')
axs[1].set_ylabel('Amplitud')
axs[1].legend()
axs[1].grid()
plt.tight_layout()
plt.show()
